<a href="https://colab.research.google.com/github/alexandre-cameron-borges/ACBorges_Clickscore_SDUDAnalytics/blob/main/ACBorges_Clickscore_SDUDAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 0) Debug
!pip install -q 'jupyter-client<8.0.0'

# 1) Cloner le repo & installer les outils
!git clone https://github.com/alexandre-cameron-borges/ACBorges_Clickscore_SDUDAnalytics.git
%cd ACBorges_Clickscore_SDUDAnalytics
!pip install -q -r requirements.txt nbstripout kaggle

# 2) Nettoyer les notebooks avant commit
!nbstripout --install

# 3) Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

Cloning into 'ACBorges_Clickscore_SDUDAnalytics'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 21 (delta 6), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (21/21), 30.06 KiB | 3.01 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/ACBorges_Clickscore_SDUDAnalytics/ACBorges_Clickscore_SDUDAnalytics
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## A FAIRE QUE LA PREMIERE FOIS

# 4) Configurer l’API Kaggle
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/credentials/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 5) Créer le dossier raw
!mkdir -p /content/drive/MyDrive/data/raw

# 6) Télécharger & dézipper les jeux Kaggle
!kaggle datasets download -d amananandrai/clickbait-dataset \
    -p /content/drive/MyDrive/data/raw --unzip
!kaggle datasets download -d marius2303/ad-click-prediction-dataset \
    -p /content/drive/MyDrive/data/raw --unzip
!kaggle datasets download -d souvik1618/ad-click-prediction-dataset \
    -p /content/drive/MyDrive/data/raw --unzip

# 7) Télécharger + unzip Webis Clickbait Corpus 2017
!wget -q -O /content/drive/MyDrive/data/raw/clickbait17-train-170630.zip "https://zenodo.org/record/5530410/files/clickbait17-train-170630.zip?download=1"
!unzip -q /content/drive/MyDrive/data/raw/clickbait17-train-170630.zip -d /content/drive/MyDrive/data/raw/webis_clickbait
!rm /content/drive/MyDrive/data/raw/clickbait17-train-170630.zip

In [ ]:
import pandas as pd

# Chemin vers le CSV amandarai clickbaitdataset dans ton Drive
pathtableclickbait_a = "/content/drive/MyDrive/data/raw/clickbait_data.csv"
tableclickbait_a = pd.read_csv(pathtableclickbait_a)

# Changement de la colonne de clickbait en Clickbait et des valeurs 0 en no-clickbait, 1 en clickbait et de headline en Texte
tableclickbait_a['clickbait'] = tableclickbait_a['clickbait'].map({1: 'clickbait', 0: 'no-clickbait'})
tableclickbait_a.rename(columns={'clickbait': 'Clickbait'}, inplace=True)
tableclickbait_a.rename(columns={'headline': 'Texte'}, inplace=True)


# Aperçu rapide
print("Shape :", tableclickbait_a.shape)
display(tableclickbait_a.head(8))

# Comptage des nulls et types
display(tableclickbait_a.info())
display(tableclickbait_a.isnull().sum())

# Répartition clickbait vs non-clickbait
counts = tableclickbait_a['Clickbait'].value_counts()
print(counts)



Shape : (32000, 2)


,Texte,Clickbait
0,Should I Get Bings,clickbait
1,Which TV Female Friend Group Do You Belong In,clickbait
2,"The New ""Star Wars: The Force Awakens"" Trailer...",clickbait
3,"This Vine Of New York On ""Celebrity Big Brothe...",clickbait
4,A Couple Did A Stunning Photo Shoot With Their...,clickbait
5,How To Flirt With Queer Girls Without Making A...,clickbait
6,32 Cute Things To Distract From Your Awkward T...,clickbait
7,If Disney Princesses Were From Florida,clickbait


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32000 entries, 0 to 31999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Texte      32000 non-null  object
 1   Clickbait  32000 non-null  object
dtypes: object(2)
memory usage: 500.1+ KB


None

,0
Texte,0
Clickbait,0


Clickbait
no-clickbait    16001
clickbait       15999
Name: count, dtype: int64


In [ ]:
# Chemin vers le CSV marius clickbaitdataset dans ton Drive
pathtableclickbait_b = "/content/drive/MyDrive/data/raw/ad_click_dataset.csv"
tableclickbait_b = pd.read_csv(pathtableclickbait_b)

# Nettoyage des NaN et des Non-Binaire
tableclickbait_b = tableclickbait_b[tableclickbait_b['gender'].isin(['Male', 'Female'])]
tableclickbait_b = tableclickbait_b.dropna(subset=['age'])

# Changement colonne 'click' en 'clickbait' et les valeurs 0 en no-clickbait, 1 en clickbait
tableclickbait_b['click'] = tableclickbait_b['click'].map({1: 'clickbait', 0: 'no-clickbait'})
tableclickbait_b.rename(columns={'click': 'Clickbait'}, inplace=True)

# Aperçu rapide
print("Shape :", tableclickbait_b.shape)
display(tableclickbait_b.head(8))

# Comptage des nulls et types
display(tableclickbait_b.info())
display(tableclickbait_b.isnull().sum())

# Répartition clickbait vs non-clickbait
counts = tableclickbait_b['Clickbait'].value_counts()
print(counts)


Shape : (1972, 9)


,id,full_name,age,gender,device_type,ad_position,browsing_history,time_of_day,Clickbait
3,5418,User5418,34.0,Male,NaN,NaN,Entertainment,Evening,clickbait
6,7808,User7808,26.0,Female,Desktop,Top,NaN,NaN,clickbait
7,5065,User5065,40.0,Male,Mobile,Side,NaN,Evening,no-clickbait
17,188,User188,56.0,Female,Tablet,Bottom,News,Morning,clickbait
25,4890,User4890,43.0,Male,Tablet,Bottom,Education,Afternoon,clickbait
27,5055,User5055,45.0,Male,NaN,NaN,Education,Morning,clickbait
28,6724,User6724,37.0,Female,NaN,Top,Entertainment,Evening,clickbait
31,1729,User1729,26.0,Female,Mobile,Top,NaN,Afternoon,clickbait


<class 'pandas.core.frame.DataFrame'>
Index: 1972 entries, 3 to 9999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                1972 non-null   int64  
 1   full_name         1972 non-null   object 
 2   age               1972 non-null   float64
 3   gender            1972 non-null   object 
 4   device_type       1587 non-null   object 
 5   ad_position       1562 non-null   object 
 6   browsing_history  1098 non-null   object 
 7   time_of_day       1585 non-null   object 
 8   Clickbait         1972 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 154.1+ KB


None

,0
id,0
full_name,0
age,0
gender,0
device_type,385
ad_position,410
browsing_history,874
time_of_day,387
Clickbait,0


Clickbait
clickbait       1276
no-clickbait     696
Name: count, dtype: int64


In [ ]:
# Chemin vers le CSV souvik clickbaitdataset dans ton Drive
pathtableclickbait_c = "/content/drive/MyDrive/data/raw/advertising.csv"
tableclickbait_c = pd.read_csv(pathtableclickbait_c)

# Changement colonne Male en Gender et les valeurs 0 en Female, 1 en Male et Ad Topic Line en Texte et Age en age
tableclickbait_c['Male'] = tableclickbait_c['Male'].map({1: 'Male', 0: 'Female'})
tableclickbait_c.rename(columns={'Male': 'gender'}, inplace=True)
tableclickbait_c.rename(columns={'Age': 'age'}, inplace=True)
tableclickbait_c.rename(columns={'Ad Topic Line': 'Texte'}, inplace=True)


# Changement colonne 'clicked on ad' en 'clickbait' et les valeurs 0 en no-clickbait, 1 en clickbait
tableclickbait_c['Clicked on Ad'] = tableclickbait_c['Clicked on Ad'].map({1: 'clickbait', 0: 'no-clickbait'})
tableclickbait_c.rename(columns={'Clicked on Ad': 'Clickbait'}, inplace=True)

# Aperçu rapide
print("Shape :", tableclickbait_c.shape)
display(tableclickbait_c.head(8))

# Comptage des nulls et types
display(tableclickbait_c.info())
display(tableclickbait_c.isnull().sum())

# Répartition clickbait vs non-clickbait
counts = tableclickbait_c['Clickbait'].value_counts()
print(counts)

Shape : (1000, 10)


,Daily Time Spent on Site,age,Area Income,Daily Internet Usage,Texte,City,gender,Country,Timestamp,Clickbait
0,68.95,35,61833.90,256.09,Cloned 5thgeneration orchestration,Wrightburgh,Female,Tunisia,2016-03-27 00:53:11,no-clickbait
1,80.23,31,68441.85,193.77,Monitored national standardization,West Jodi,Male,Nauru,2016-04-04 01:39:02,no-clickbait
2,69.47,26,59785.94,236.50,Organic bottom-line service-desk,Davidton,Female,San Marino,2016-03-13 20:35:42,no-clickbait
3,74.15,29,54806.18,245.89,Triple-buffered reciprocal time-frame,West Terrifurt,Male,Italy,2016-01-10 02:31:19,no-clickbait
4,68.37,35,73889.99,225.58,Robust logistical utilization,South Manuel,Female,Iceland,2016-06-03 03:36:18,no-clickbait
5,59.99,23,59761.56,226.74,Sharable client-driven software,Jamieberg,Male,Norway,2016-05-19 14:30:17,no-clickbait
6,88.91,33,53852.85,208.36,Enhanced dedicated support,Brandonstad,Female,Myanmar,2016-01-28 20:59:32,no-clickbait
7,66.00,48,24593.33,131.76,Reactive local challenge,Port Jefferybury,Male,Australia,2016-03-07 01:40:15,clickbait


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Daily Time Spent on Site  1000 non-null   float64
 1   age                       1000 non-null   int64  
 2   Area Income               1000 non-null   float64
 3   Daily Internet Usage      1000 non-null   float64
 4   Texte                     1000 non-null   object 
 5   City                      1000 non-null   object 
 6   gender                    1000 non-null   object 
 7   Country                   1000 non-null   object 
 8   Timestamp                 1000 non-null   object 
 9   Clickbait                 1000 non-null   object 
dtypes: float64(3), int64(1), object(6)
memory usage: 78.3+ KB


None

,0
Daily Time Spent on Site,0
age,0
Area Income,0
Daily Internet Usage,0
Texte,0
City,0
gender,0
Country,0
Timestamp,0
Clickbait,0


Clickbait
no-clickbait    500
clickbait       500
Name: count, dtype: int64


In [ ]:
# 1) Chemin vers fichier webis JSONL & 2) Charger le JSONL
pathtableclickbait_d = "/content/drive/MyDrive/data/raw/webis_clickbait/clickbait17-validation-170630/truth.jsonl"
pathtableclickbait_d2 = "/content/drive/MyDrive/data/raw/webis_clickbait/clickbait17-validation-170630/instances.jsonl"
tableclickbait_d = pd.read_json(pathtableclickbait_d, lines=True)
tableclickbait_d2 = pd.read_json(pathtableclickbait_d2, lines=True)

# Changement colonne 'truthClass' en 'clickbait' et targetTitle en Texte
tableclickbait_d.rename(columns={'truthClass': 'Clickbait'}, inplace=True)
tableclickbait_d2.rename(columns={'targetTitle': 'Texte'}, inplace=True)


# Aperçu rapide
print("Shape :", tableclickbait_d.shape)
display(tableclickbait_d.head(8))

print("Shape :", tableclickbait_d2.shape)
display(tableclickbait_d2.head(8))

# Comptage des nulls et types
display(tableclickbait_d.info())
display(tableclickbait_d.isnull().sum())

display(tableclickbait_d2.info())
display(tableclickbait_d2.isnull().sum())

# Répartition clickbait vs non-clickbait
counts = tableclickbait_d['Clickbait'].value_counts()
print(counts)

Shape : (19538, 6)


,truthJudgments,truthMean,id,Clickbait,truthMedian,truthMode
0,"[1.0, 1.0, 1.0, 1.0, 1.0]",1.000000,858464162594172928,clickbait,1.000000,1.000000
1,"[0.33333333330000003, 0.0, 0.33333333330000003...",0.133333,858462320779026432,no-clickbait,0.000000,0.000000
2,"[0.33333333330000003, 0.6666666666000001, 1.0,...",0.400000,858460992073863168,no-clickbait,0.333333,0.000000
3,"[0.0, 0.6666666666000001, 0.0, 0.3333333333000...",0.266667,858459539296980992,no-clickbait,0.333333,0.333333
4,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,858455355948384256,no-clickbait,0.000000,0.000000
5,"[0.0, 0.6666666666000001, 0.33333333330000003,...",0.200000,858453552892694528,no-clickbait,0.000000,0.000000
6,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,858450499921731584,no-clickbait,0.000000,0.000000
7,"[0.0, 0.0, 0.0, 0.6666666666000001, 0.0]",0.133333,858449569750896640,no-clickbait,0.000000,0.000000


Shape : (19538, 9)


,postMedia,postText,id,targetCaptions,targetParagraphs,Texte,postTimestamp,targetKeywords,targetDescription
0,[],[UK’s response to modern slavery leaving victi...,858462320779026432,[modern-slavery-rex.jpg],[Thousands of modern slavery victims have not ...,‘Inexcusable’ failures in UK’s response to mod...,Sat Apr 29 23:25:41 +0000 2017,"modern slavery, Department For Work And Pensio...",“Inexcusable” failures in the UK’s system for ...
1,[],[this is good],858421020331560960,"[In this July 1, 2010 file photo, Dr. Charmain...",[President Donald Trump has appointed the pro-...,Donald Trump Appoints Pro-Life Advocate as Ass...,Sat Apr 29 20:41:34 +0000 2017,"Americans United for Life, Dr. Charmaine Yoest...",President Donald Trump has appointed pro-life ...
2,[],"[The ""forgotten"" Trump roast: Relive his bruta...",858368123753435136,[President Trump will not attend this year's W...,[When the White House correspondents’ dinner i...,The ‘forgotten’ Trump roast: Relive his brutal...,Sat Apr 29 17:11:23 +0000 2017,"trump whcd, whcd, white house correspondents d...",President Trump won't be at this year's White ...
3,[],[Meet the happiest #dog in the world!],858323428260139008,"[Maru , Maru, Maru, Maru, Maru]",[Adorable is probably an understatement. This ...,"Meet The Happiest Dog In The World, Maru The H...",Sat Apr 29 14:13:46 +0000 2017,"Maru, husky, dogs, pandas, furball, instagram","The article is about Maru, a husky dog who has..."
4,[],[Tokyo's subway is shut down amid fears over a...,858283602626347008,[All nine lines of Tokyo's subway system were ...,[One of Tokyo's major subways systems says it ...,Tokyo's subway is shut down amid fears over an...,Sat Apr 29 11:35:31 +0000 2017,"Tokyo,subway,shut,fears,North,Korean,attack","The temporary suspension, which lasted ten min..."
5,[],[Ban lifted on Madrid doping laboratory],858224473597779968,"[Samples in an anti-doping laboratory, Anthony...","[Share this with, Madrid's Anti-Doping Laborat...",World Anti-Doping Agency lifts ban on Madrid l...,Sat Apr 29 07:40:34 +0000 2017,,Madrid's Anti-Doping Laboratory has its suspen...
6,[],"[Despite the ‘Yuck Factor,’ Leeches Are Big in...",858222698367000576,"[The New York Times, Basic, All Access, Home D...",[MOSCOW — They are small as physician assistan...,"Despite the ‘Yuck Factor,’ Leeches Are Big in ...",Sat Apr 29 07:33:30 +0000 2017,"Drugs (Pharmaceuticals),Medical Devices,Russia...",Russians use about 10 million leeches a year t...
7,[],[#China and #Pakistan have cemented their poli...,858191667739987968,[],[China and Pakistan have cemented their politi...,China And Pakistan Use Biryani To Take Their F...,Sat Apr 29 05:30:12 +0000 2017,"Pakistan, China, Biryani, CPEC, china pakistan...",China And Pakistan Use Biryani To Take Their F...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19538 entries, 0 to 19537
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   truthJudgments  19538 non-null  object 
 1   truthMean       19538 non-null  float64
 2   id              19538 non-null  int64  
 3   Clickbait       19538 non-null  object 
 4   truthMedian     19538 non-null  float64
 5   truthMode       19538 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 916.0+ KB


...

,0
truthJudgments,0
truthMean,0
id,0
Clickbait,0
truthMedian,0
truthMode,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19538 entries, 0 to 19537
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   postMedia          19538 non-null  object
 1   postText           19538 non-null  object
 2   id                 19538 non-null  int64 
 3   targetCaptions     19538 non-null  object
 4   targetParagraphs   19538 non-null  object
 5   Texte              19538 non-null  object
 6   postTimestamp      19538 non-null  object
 7   targetKeywords     19538 non-null  object
 8   targetDescription  19538 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.3+ MB


None

,0
postMedia,0
postText,0
id,0
targetCaptions,0
targetParagraphs,0
Texte,0
postTimestamp,0
targetKeywords,0
targetDescription,0


Clickbait
no-clickbait    14777
clickbait        4761
Name: count, dtype: int64


In [ ]:
#Filtre des colonnes utiles

cols_a   = ['Texte', 'Clickbait']
cols_b   = ['age', 'gender', 'Clickbait']
cols_c   = ['age', 'gender', 'Texte', 'Clickbait']
cols_d   = ['Clickbait', 'id']
cols_d2  = ['Texte', 'postMedia', 'id']

tableclickbait_a  = tableclickbait_a[cols_a]
tableclickbait_b  = tableclickbait_b[cols_b]
tableclickbait_c  = tableclickbait_c[cols_c]
tableclickbait_d  = tableclickbait_d[cols_d]
tableclickbait_d2 = tableclickbait_d2[cols_d2]

display(tableclickbait_a.head(5))
display(tableclickbait_b.head(5))
display(tableclickbait_c.head(5))
display(tableclickbait_d.head(5))
display(tableclickbait_d2.head(5))

,Texte,Clickbait
0,Should I Get Bings,clickbait
1,Which TV Female Friend Group Do You Belong In,clickbait
2,"The New ""Star Wars: The Force Awakens"" Trailer...",clickbait
3,"This Vine Of New York On ""Celebrity Big Brothe...",clickbait
4,A Couple Did A Stunning Photo Shoot With Their...,clickbait


,age,gender,Clickbait
3,34.0,Male,clickbait
6,26.0,Female,clickbait
7,40.0,Male,no-clickbait
17,56.0,Female,clickbait
25,43.0,Male,clickbait


,age,gender,Texte,Clickbait
0,35,Female,Cloned 5thgeneration orchestration,no-clickbait
1,31,Male,Monitored national standardization,no-clickbait
2,26,Female,Organic bottom-line service-desk,no-clickbait
3,29,Male,Triple-buffered reciprocal time-frame,no-clickbait
4,35,Female,Robust logistical utilization,no-clickbait


,Clickbait,id
0,clickbait,858464162594172928
1,no-clickbait,858462320779026432
2,no-clickbait,858460992073863168
3,no-clickbait,858459539296980992
4,no-clickbait,858455355948384256


,Texte,postMedia,id
0,‘Inexcusable’ failures in UK’s response to mod...,[],858462320779026432
1,Donald Trump Appoints Pro-Life Advocate as Ass...,[],858421020331560960
2,The ‘forgotten’ Trump roast: Relive his brutal...,[],858368123753435136
3,"Meet The Happiest Dog In The World, Maru The H...",[],858323428260139008
4,Tokyo's subway is shut down amid fears over an...,[],858283602626347008


In [ ]:
#Fusion de D & D2 (Webis)

tableclickbait_dd2 = pd.merge(
    tableclickbait_d,
    tableclickbait_d2,
    on="id",
    how="inner"
)

#Filtre
cols_dd2  = ['Texte', 'Clickbait']
tableclickbait_dd2 = tableclickbait_dd2[cols_dd2]


display(tableclickbait_dd2.head(5))
display(tableclickbait_dd2.info())


,Texte,Clickbait
0,Facts That Will Be Truly Upsetting To '90s Girls,clickbait
1,‘Inexcusable’ failures in UK’s response to mod...,no-clickbait
2,Inside Kim Jong-un's camps of death: Former No...,no-clickbait
3,Trump Has Flip-Flopped. But His Supporters Are...,no-clickbait
4,Donald Trump said being US president was harde...,no-clickbait


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19538 entries, 0 to 19537
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Texte      19538 non-null  object
 1   Clickbait  19538 non-null  object
dtypes: object(2)
memory usage: 305.4+ KB


None

In [ ]:
#Fusion de B & C

tableclickbait_bc = tableclickbait_b.merge(
    tableclickbait_c,
    on=["age", "gender", "Clickbait"],
    how="outer",
    suffixes=("_b", "_c")
)

display(tableclickbait_bc.head(5))
display(tableclickbait_bc.info())

,age,gender,Clickbait,Texte
0,18.0,Female,clickbait,NaN
1,18.0,Female,clickbait,NaN
2,18.0,Female,clickbait,NaN
3,18.0,Female,clickbait,NaN
4,18.0,Female,clickbait,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11229 entries, 0 to 11228
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        11229 non-null  float64
 1   gender     11229 non-null  object 
 2   Clickbait  11229 non-null  object 
 3   Texte      10802 non-null  object 
dtypes: float64(1), object(3)
memory usage: 351.0+ KB


None

In [ ]:
# Fusion des tables A & D D2

tableclickbait_add2 = tableclickbait_a.merge(
    tableclickbait_dd2,
    on=["Texte", "Clickbait"],
    how="outer",
    suffixes=("_a", "_dd2")
)

display(tableclickbait_add2.head(10))
display(tableclickbait_add2.info())

,Texte,Clickbait
0,"!Sdrawkcab: Missy Elliott, the Beatles and the...",no-clickbait
1,"""Apprentice"" contestant sues Trump for defamation",no-clickbait
2,"""Big morale boost"": George H.W. Bush tweets im...",no-clickbait
3,"""Bring it on"": Students sue Trump administrati...",no-clickbait
4,"""Frat Lives Matter"" spray-painted on UConn cam...",no-clickbait
5,"""Giant hurricane"" on Saturn: First images from...",no-clickbait
6,"""God made me bulletproof,"" oft-shot rapper Yun...",no-clickbait
7,"""Just stop daddy"" children scream in 911 call ...",no-clickbait
8,"""Last Tango in Paris"" rape scene revelation sp...",no-clickbait
9,"""Moonlight"" writer on its origin and cirtical ...",no-clickbait


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51538 entries, 0 to 51537
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Texte      51538 non-null  object
 1   Clickbait  51538 non-null  object
dtypes: object(2)
memory usage: 805.4+ KB


None

In [ ]:
# Fusion de toutes les tables A B C D D2

tableclickbait_abcdd2 = tableclickbait_add2.merge(
    tableclickbait_bc,
    on=["Texte", "Clickbait"],
    how="outer",
    suffixes=("_add2", "_bc")
)

display(tableclickbait_abcdd2.head(20))
display(tableclickbait_abcdd2.info())

,Texte,Clickbait,age,gender
0,"!Sdrawkcab: Missy Elliott, the Beatles and the...",no-clickbait,NaN,NaN
1,"""Apprentice"" contestant sues Trump for defamation",no-clickbait,NaN,NaN
2,"""Big morale boost"": George H.W. Bush tweets im...",no-clickbait,NaN,NaN
3,"""Bring it on"": Students sue Trump administrati...",no-clickbait,NaN,NaN
4,"""Frat Lives Matter"" spray-painted on UConn cam...",no-clickbait,NaN,NaN
5,"""Giant hurricane"" on Saturn: First images from...",no-clickbait,NaN,NaN
6,"""God made me bulletproof,"" oft-shot rapper Yun...",no-clickbait,NaN,NaN
7,"""Just stop daddy"" children scream in 911 call ...",no-clickbait,NaN,NaN
8,"""Last Tango in Paris"" rape scene revelation sp...",no-clickbait,NaN,NaN
9,"""Moonlight"" writer on its origin and cirtical ...",no-clickbait,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62767 entries, 0 to 62766
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Texte      62340 non-null  object 
 1   Clickbait  62767 non-null  object 
 2   age        11229 non-null  float64
 3   gender     11229 non-null  object 
dtypes: float64(1), object(3)
memory usage: 1.9+ MB


None

In [ ]:
# Prétraitement des données

df = tableclickbait_abcdd2.dropna(subset=['Texte']).copy()
df['label'] = (df['Clickbait']=='clickbait').astype(int)

med = df['age'].median()
df['age'] = df['age'].fillna(med)
df['age_norm'] = (df['age'] - med) / (df['age'].max() - med)

df['gender'] = df['gender'].fillna('Unknown')
df['gender_id'] = df['gender'].astype('category').cat.codes



In [ ]:
# Split train / validation

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, stratify=df['label'], test_size=0.2, random_state=42)

In [ ]:
# Définir le dataset

from torch.utils.data import Dataset
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

class CB_Dataset(Dataset):
    def __init__(self, df):
        self.texts = df["Texte"].tolist()
        self.ages  = df["age_norm"].tolist()
        self.genders = df["gender_id"].tolist()
        self.labels = df["label"].tolist()
    def __len__(self): return len(self.labels)
    def __getitem__(self, i):
        enc = tokenizer(self.texts[i], padding="max_length", truncation=True,
                        max_length=128, return_tensors="pt")
        return {
          "input_ids": enc.input_ids.squeeze(0),
          "attention_mask": enc.attention_mask.squeeze(0),
          "age": torch.tensor(self.ages[i], dtype=torch.float),
          "gender": torch.tensor(self.genders[i], dtype=torch.long),
          "label": torch.tensor(self.labels[i], dtype=torch.float)
        }


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
# DataLoaders


from torch.utils.data import DataLoader
train_loader = DataLoader(CB_Dataset(train_df), batch_size=16, shuffle=True)
val_loader   = DataLoader(CB_Dataset(val_df),   batch_size=16)


In [ ]:
# Architecture du modèle
import torch
import torch.nn as nn, torch.nn.functional as F
from transformers import BertModel
from transformers import get_linear_schedule_with_warmup


class ClickbaitModel(nn.Module):
    def __init__(self, n_genders):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-multilingual-cased")
        self.age_fc = nn.Linear(1, 16)
        self.gender_emb = nn.Embedding(n_genders, 8)
        hid = self.bert.config.hidden_size + 16 + 8
        self.head = nn.Sequential(
            nn.Linear(hid, 64), nn.ReLU(), nn.Dropout(0.1),
            nn.Linear(64, 1)
        )
    def forward(self, input_ids, attention_mask, age, gender):
        out = self.bert(input_ids, attention_mask).pooler_output
        a = F.relu(self.age_fc(age.unsqueeze(1)))
        g = self.gender_emb(gender)
        x = torch.cat([out, a, g], dim=1)
        return self.head(x).squeeze(-1)
model = ClickbaitModel(n_genders=df['gender_id'].nunique())


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
criterion = nn.BCEWithLogitsLoss(
  pos_weight=torch.tensor([df['label'].value_counts(normalize=True)[0] /
                           df['label'].value_counts(normalize=True)[1]])
)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Define num_epochs here
num_epochs = 4  # You can change this to your desired number of epochs

total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(
  optimizer, num_warmup_steps=0.1*total_steps, num_training_steps=total_steps
)


In [ ]:
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        # Exclude 'label' from the input to the model
        logits = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'],
                       age=batch['age'], gender=batch['gender'])
        loss = criterion(logits, batch['label'])
        loss.backward()
        optimizer.step(); scheduler.step()

    # Évaluation à la fin de l’époque
    model.eval()
    preds, truths = [], []
    with torch.no_grad():
        for b in val_loader:
            # Exclude 'label' from the input to the model
            logits = model(input_ids=b['input_ids'], attention_mask=b['attention_mask'],
                           age=b['age'], gender=b['gender'])
            preds += torch.sigmoid(logits).round().tolist()
            truths += b['label'].tolist()
    val_f1 = f1_score(truths, preds)
    print(f"Epoch {epoch+1}: val F1 = {val_f1:.3f}")

if val_f1 > best_f1:
        best_f1, patience = val_f1, 0
        torch.save(model.state_dict(), "best_cb_model.pt")
else:
        patience += 1
        if patience >= 2:
            print("Early stopping")
            break

# 6) Charger le meilleur modèle
model.load_state_dict(torch.load("best_cb_model.pt", map_location=device))
model.eval()

KeyboardInterrupt: 

In [ ]:
%cd ACBorges_Clickscore_SDUDAnalytics
!pwd
!ls -1

%%bash
cd /content/ACBorges_Clickscore_SDUDAnalytics
cat << 'EOF' > app.py
import streamlit as st
from predict import predict_cb
# -- UI --
st.title("Détection de Clickbait")
texte = st.text_area("Texte publicitaire")
age   = st.slider("Âge cible", 18.0, 99.0, 30.0)
genre = st.selectbox("Genre cible", ["Male","Female","Unknown"])
gender_map = {"Male":0,"Female":1,"Unknown":2}
if st.button("Prédire"):
    age_norm = (age - MED) / (MAX_AGE - MED)
    prob = predict_cb(texte, age_norm, gender_map[genre])
    label = "Clickbait" if prob>=0.5 else "Non-Clickbait"
    st.metric("Probabilité", f"{prob:.1%}")
    st.success(f"Résultat : {label}")
EOF

!ls -1

streamlit run app.py
